Imports

In [66]:
from src.smallcnn import SmallCNN
from src.dataset import FMADataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from src.audio_processing import audio_to_mel_tensor
import os
import librosa
import numpy as np


Maaping des genres

In [ ]:
genres_df = pd.read_csv("fma_metadata/genres.csv")  # ton tableau avec genre_id,title,etc.

genre_ids = sorted(genres_df["genre_id"].tolist())
genre_id_to_idx = {gid: i for i, gid in enumerate(genre_ids)}
print(genre_id_to_idx)
num_classes = len(genre_id_to_idx)
print(f"Nombre de classes: {num_classes}")


{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 25: 22, 26: 23, 27: 24, 30: 25, 31: 26, 32: 27, 33: 28, 36: 29, 37: 30, 38: 31, 41: 32, 42: 33, 43: 34, 45: 35, 46: 36, 47: 37, 49: 38, 53: 39, 58: 40, 63: 41, 64: 42, 65: 43, 66: 44, 70: 45, 71: 46, 74: 47, 76: 48, 77: 49, 79: 50, 81: 51, 83: 52, 85: 53, 86: 54, 88: 55, 89: 56, 90: 57, 92: 58, 94: 59, 97: 60, 98: 61, 100: 62, 101: 63, 102: 64, 103: 65, 107: 66, 109: 67, 111: 68, 113: 69, 117: 70, 118: 71, 125: 72, 130: 73, 137: 74, 138: 75, 166: 76, 167: 77, 169: 78, 170: 79, 171: 80, 172: 81, 173: 82, 174: 83, 175: 84, 176: 85, 177: 86, 178: 87, 179: 88, 180: 89, 181: 90, 182: 91, 183: 92, 184: 93, 185: 94, 186: 95, 187: 96, 188: 97, 189: 98, 214: 99, 224: 100, 232: 101, 236: 102, 240: 103, 247: 104, 250: 105, 267: 106, 286: 107, 296: 108, 297: 109, 311: 110, 314: 111, 322: 112, 337: 113, 359: 114, 360: 115, 361: 116, 362: 117, 

In [ ]:
# root = "fma_small"
# file_paths = []

# for dirpath, dirnames, filenames in os.walk(root):
#     for filename in filenames:
#         if filename.endswith(".mp3"):  # important
#             full_path = os.path.join(dirpath, filename)
#             file_paths.append(full_path)


In [80]:
import ast  # pour convertir les listes "[21, 103]" en vraies listes python

tracks_df = pd.read_csv("fma_metadata/tracks.csv", header=[0,1], index_col=0)

genres_raw = tracks_df[("track", "genres")]  # Series: index = track_id
track_genre_id_map = {}                     # mapping final : track_id -> genre_id

for tid, string_list in genres_raw.items():
    try:
        genre_ids = ast.literal_eval(string_list)  # "[21]" -> [21]
        if len(genre_ids) > 0:
            track_genre_id_map[tid] = genre_ids[0]  # premier genre_id
    except:
        pass

track_genre_ids = []

for path in file_paths:
    filename = os.path.basename(path)        # "000002.mp3"
    track_id = int(filename.replace(".mp3", ""))  # 2

    if track_id in track_genre_id_map:
        track_genre_ids.append(track_genre_id_map[track_id])
    else:
        # si jamais un fichier n’a pas de genre dans tracks.csv
        track_genre_ids.append(None)  

print(len(track_genre_ids) == len(file_paths)
)

True


In [ ]:
# unique_genre_ids = sorted(set(track_genre_ids))

# genre_id_to_idx = {gid: i for i, gid in enumerate(unique_genre_ids)}
# idx_to_genre_id = {i: gid for gid, i in genre_id_to_idx.items()}


In [70]:
# track_genre_ids : liste des genre_id associés à chaque fichier (par ex. [21, 21, 10, 76, ...])

unique_genre_ids = sorted(set(track_genre_ids))
genre_id_to_idx = {gid: i for i, gid in enumerate(unique_genre_ids)}
idx_to_genre_id = {i: gid for gid, i in genre_id_to_idx.items()}

num_classes = len(unique_genre_ids)
model = SmallCNN(n_mels=128, num_classes=num_classes).to(device)


In [71]:
import pandas as pd

tracks_df = pd.read_csv("fma_metadata/tracks.csv", header=[0,1], index_col=0)

print(tracks_df.head())
print(tracks_df.columns)
print(tracks_df.index)


            album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
20              0  2008-11-26 01:45:05  2009-01-06 00:00:00      NaN   

                                                                          \
         favorites id                                information listens   
track_id                                                                   
2                4  1                                    <p></p>    6073   
3                4  1                                    <p></p>    6073   
5                4  1                      

In [72]:
tracks_df = pd.read_csv("fma_metadata/tracks.csv", header=[0, 1], index_col=0)
genre_top = tracks_df[("track", "genre_top")]   # Série pandas
# mapping : track_id (index) → genre_top (str)
track_genre_top_map = genre_top.to_dict()

In [73]:
import pandas as pd

# Charger le CSV des genres
genres_df = pd.read_csv("fma_metadata/genres.csv")  # le texte que tu as collé

# liste de tous les genre_id
genre_ids = sorted(genres_df["genre_id"].tolist())

# mapping pour l'entraînement
genre_id_to_idx = {gid: i for i, gid in enumerate(genre_ids)}

# mapping inverse pour interpréter les prédictions
idx_to_genre_id = {i: gid for gid, i in genre_id_to_idx.items()}

# si tu veux aussi le titre :
genre_id_to_title = {
    row["genre_id"]: row["title"]
    for _, row in genres_df.iterrows()
}

num_classes = len(genre_ids)
print("Nombre de genres:", num_classes)



Nombre de genres: 163


Préparation des données

In [74]:
root = "fma_small"
file_paths = []
labels = []

for dirpath, dirnames, filenames in os.walk(root):
    for filename in filenames:
        if filename.endswith(".mp3"):   # important : ignorer les autres fichiers
            full_path = os.path.join(dirpath, filename)

            # label = nom du dossier parent (ex: "000")
            label = os.path.basename(dirpath)

            file_paths.append(full_path)
            labels.append(int(label))

valid_paths = []
valid_genre_ids = []

for path, gid in zip(file_paths, track_genre_ids):
    mel, sr = audio_to_mel_tensor(path)
    if mel is None:
        print(f"On ignore le fichier corrompu : {path}")
        continue
    valid_paths.append(path)
    valid_genre_ids.append(gid)

file_paths = valid_paths
track_genre_ids = valid_genre_ids


/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/Users/remi-moustamsik/Documents/projet_lock/venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Erreur de chargement fma_small/133/133297.mp3: 
On ignore le fichier corrompu : fma_small/133/133297.mp3


[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3360) too large for available bit count (3240)
[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3328) too large for available bit count (3240)
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1349] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
/Users/remi-moustamsik/Documents/projet_lock/venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
No

Erreur de chargement fma_small/099/099134.mp3: 
On ignore le fichier corrompu : fma_small/099/099134.mp3


/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/Users/remi-moustamsik/Documents/projet_lock/venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Erreur de chargement fma_small/108/108925.mp3: 
On ignore le fichier corrompu : fma_small/108/108925.mp3


[src/libmpg123/layer3.c:INT123_do_layer3():1844] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!


Préparation du modèle

In [ ]:
# Pour GPU classique
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Pour gpu Apple MPS
# Permet d'être presque 10 fois plus rapide que sur CPU

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✓ Entraînement sur GPU Apple (MPS)")
else:
    device = torch.device("cpu")
    print("⚠️ Entraînement sur CPU")


model = SmallCNN(n_mels=128, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model.to(device)


✓ Entraînement sur GPU Apple (MPS)


SmallCNN(
  (conv_block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (conv_block4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  

Entrainement

In [76]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device)   # (B, 1, 128, T)
        labels = labels.to(device)   # (B,)
        
        optimizer.zero_grad()
        
        logits, _ = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

dataset = FMADataset(file_paths, labels, genre_id_to_idx)


Evaluation

In [78]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            logits, probs = model(inputs)
            loss = criterion(logits, labels)
            
            running_loss += loss.item() * inputs.size(0)
            
            # prédiction = classe avec probabilité max
            _, preds = torch.max(probs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy



In [79]:
NUM_EPOCHS = 2  # choisis le nombre d'epochs


dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

dataset = FMADataset(file_paths, track_genre_ids, max_len=3000, genre_id_to_idx=genre_id_to_idx)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)


for epoch in range(NUM_EPOCHS):
    # ---- TRAIN ----
    train_loss = train_one_epoch(
        model=model,
        dataloader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device
    )
    
    # ---- TEST ----
    test_loss, test_acc = evaluate(
        model=model,
        dataloader=test_loader,
        criterion=criterion,
        device=device
    )

    print(
        f"Epoch {epoch+1}/{NUM_EPOCHS} "
        f"- train loss: {train_loss:.4f} | test loss: {test_loss:.4f} | test acc: {test_acc*100:.2f}%"
    )


[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 22401.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1349] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
/Users/remi-moustamsik/Documents/projet_lock/venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3360) too large for available bit count (3240)
[src/libmpg123/layer3.c:INT123_do_layer3()

Epoch 1/2 - train loss: 3.6832 | test loss: 3.6277 | test acc: 8.01%


[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1349] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3360) too large for available bit count (3240)
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 22401.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1349] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3328) too large for available bit count (3240)
[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: 

Epoch 2/2 - train loss: 3.6136 | test loss: 3.6509 | test acc: 14.45%
